In [1]:
import aocd

In [2]:
data = aocd.get_data(day=9, year=2024)
disk_map = [int(n) for n in data]

In [3]:
def files(disk_map):
    for n, size in enumerate(disk_map):
        if n % 2:
            yield None, size
        else:
            yield n // 2, size

def blocks(files):
    for file_id, file_size in files:
        for _ in range(file_size):
            if file_id != None:
                yield file_id, 1


def compact(disk_map):
    num_blocks = sum(disk_map[::2])
    from_end = blocks(reversed(list(files(disk_map))))
    file_id = 0

    n = 0
    while True:
        for _ in range(disk_map[2*file_id]):
            yield (file_id, 1)
            n += 1
            if n >= num_blocks:
                return
        for _ in  range(disk_map[2*file_id+1]):
            yield next(from_end)
            n += 1
            if n >= num_blocks:
                return
        file_id += 1

def checksum(files):
    sum_ = 0
    block_index = 0
    for file_id, file_size in files:
        for _ in range(file_size):
            sum_ += block_index * (file_id if file_id else 0)
            block_index += 1
    return sum_


In [4]:
part1 = checksum(compact(disk_map))
part1, part1 == 6421128769094

(6421128769094, True)

In [5]:
old_layout = list(files(disk_map))
new_layout = list(files(disk_map))
old_position = sum(sz for _, sz in old_layout)

for old_index, (file_id, file_size) in reversed(list(enumerate(old_layout))):
    old_position -= file_size
    if file_id == None:
        continue
    new_position = 0
    for new_index, (segment_id, segment_size) in enumerate(new_layout):
        new_position += segment_size
        if segment_id != None:
            continue
        if new_position > old_position:
            break
        if segment_size >= file_size:
            new_layout[new_layout.index((file_id, file_size))] = (None, file_size)
            insertion = [(file_id, file_size)]
            if gap_size := segment_size - file_size:
                insertion += [(None, gap_size)]
            new_layout[new_index:new_index+1] = insertion

            break


In [6]:
part2 = checksum(new_layout)
part2, part2 == 6448168620520

(6448168620520, True)